In [1]:
# 전처리된 파일 불러오기
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")

In [3]:
X_train.shape, y_train.shape, X_test.shape

((891, 22), (891, 1), (418, 22))

### 앙상블  모델 사용하기
 - 머신러닝에서 성능이 좋은 모델
 - 여러개의 DecisionTree를 사용하는 모델

#### (1) RandomForest 모델 사용

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
# 모델 생성
forest_model = RandomForestClassifier()

#### (2) 학습

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
forest_model.fit(X_train,y_train)

RandomForestClassifier()

#### (3) 평가

In [8]:
# 학습한 모델로 예측 후 평가
pre = forest_model.predict(X_test)

In [9]:
# 평가 결과 답안지 불러오기
sub = pd.read_csv("data/titanic/gender_submission.csv")
# 평가 결과 답안지에 예측한 값 넣기
sub['Survived'] = pre
# 예측한 값을 넣은 평가 결과 답안지 파일로 내보내기 -> kaggle
sub.to_csv('titanic3.csv', index=False)

##### 교차검증

In [10]:
from sklearn.model_selection import cross_val_score
result = cross_val_score(forest_model, X_train, y_train, cv=5)
result.mean()

0.8013558470905782

In [11]:
forest_model.fit(X_train,y_train)
pre = forest_model.predict(X_test)
sub['Survived'] = pre
sub.to_csv('titanic4.csv', index=False)

In [12]:
# 하이퍼 파라미터 튜닝
# n_estimators를 50 ~ 150까지 바꿔가면서 결과 확인
for i in range(50 , 151):
    forest_model = RandomForestClassifier(n_estimators= i)
    # 교차검증으로 평가
    result = cross_val_score(forest_model, X_train, y_train, cv=5).mean()
    print('n_estimators : {}, score : {}'.format(i,result))

n_estimators : 50, score : 0.80023225158496
n_estimators : 51, score : 0.8036030381018142
n_estimators : 52, score : 0.8024919967359236
n_estimators : 53, score : 0.8035967610319503
n_estimators : 54, score : 0.7979850605737242
n_estimators : 55, score : 0.7979976147134518
n_estimators : 56, score : 0.80248571966606
n_estimators : 57, score : 0.806973824618668
n_estimators : 58, score : 0.8069738246186681
n_estimators : 59, score : 0.8058439520431863
n_estimators : 60, score : 0.8013684012303057
n_estimators : 61, score : 0.815956311593748
n_estimators : 62, score : 0.8080911430544221
n_estimators : 63, score : 0.8002573598644153
n_estimators : 64, score : 0.8047203565375682
n_estimators : 65, score : 0.7991212102190698
n_estimators : 66, score : 0.8013621241604418
n_estimators : 67, score : 0.7991274872889335
n_estimators : 68, score : 0.8024731655263324
n_estimators : 69, score : 0.7957315924926245
n_estimators : 70, score : 0.8069487163392128
n_estimators : 71, score : 0.80023852865

KeyboardInterrupt: 

In [13]:
# (1) n_estimators를 제한 : for문 활용
# (2) max_depth를 제한 : for문 활용
# 최적의 하이퍼파리미터 조합을 찾기 위해서는 여러개의 for문이 필요
# => 그리드서치 : 한번에 여러개의 하이퍼파리미터를 튜닝해주는 함수 GridSearch()
# => GridSearchCV() : 하이퍼파라미터 튜닝 + 교차검증 

In [14]:
# 1. 하이퍼파라미터 튜닝
# GridSearchCV(모댈, 하이퍼파라미터 조합(딕셔너리), cv)

# - 관심있는 하이퍼파라미터 지정
# n_estimators = [50,70,90,110,130,150]
# max_depth = [3,4,5,6,7,8]
# max_features = [0.3, 0.4, 0.5, 0.6]

# 파라미터를 딕셔너리 형태로 저장
# key : 사용할 하이퍼파라미터, value : 범위설정
param = { 'n_estimators' : [50,70,90,110,130,150],
        'max_depth': [3,4,5,6,7,8],
        'max_features' : [0.3, 0.4, 0.5, 0.6, 0.7]}

### GridSearchCV( ) 불러오기

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
grid = GridSearchCV(RandomForestClassifier(), param, cv=5)
grid

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8],
                         'max_features': [0.3, 0.4, 0.5, 0.6, 0.7],
                         'n_estimators': [50, 70, 90, 110, 130, 150]})

In [17]:
# grid 학습
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8],
                         'max_features': [0.3, 0.4, 0.5, 0.6, 0.7],
                         'n_estimators': [50, 70, 90, 110, 130, 150]})

In [18]:
## 최적의 파라미터 조합 찾기
grid.best_params_

{'max_depth': 7, 'max_features': 0.6, 'n_estimators': 50}

In [19]:
# 최적의 파라미터일때, best score값
grid.best_score_

0.8327788588286988

In [20]:
# 자세하게 찾아보기
param = { 'n_estimators' : [48,49,50,51,52],
        'max_depth': [6,7,8,9,10],
        'max_features' : [0.6, 0.7,0.8]}
grid1 = GridSearchCV(RandomForestClassifier(), param, cv=5)
grid1.fit(X_train, y_train)
print(grid1.best_params_)
print(grid1.best_score_)

{'max_depth': 8, 'max_features': 0.7, 'n_estimators': 50}
0.8350386039796623


In [21]:
# 'max_depth': 10, 'max_features': 0.6, 'n_estimators': 49 => best 모델 생성
forest1 = RandomForestClassifier(max_depth= 10, max_features= 0.6, n_estimators=49)

In [22]:
forest1.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features=0.6, n_estimators=49)

In [23]:
# 학습한 모델로 예측 후 평가
pre = forest_model.predict(X_test)
# 평가 결과 답안지 불러오기
sub = pd.read_csv("data/titanic/gender_submission.csv")
# 평가 결과 답안지에 예측한 값 넣기
sub['Survived'] = pre
# 예측한 값을 넣은 평가 결과 답안지 파일로 내보내기 -> kaggle
sub.to_csv('gridSearchCVresult.csv', index=False)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.